# 4.4 MLP - Stochastic Gradient Descent Method

In [1]:
''' 
 Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
 Date; 3. 19. 2020 - 3. 23. 2021
 Title: Artificial Intelligence_Project 2
 Professor: Seung-Hoon Na'''
  
import numpy as np
import random
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

# mnist data

In [2]:
from sklearn import datasets
mnist = datasets.fetch_openml('mnist_784')

In [3]:
(mnist.data.shape, mnist.target.shape)

((70000, 784), (70000,))

In [4]:
np.unique(mnist.target)

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)

In [5]:
data=list(zip(mnist.data,mnist.target))
data=np.array(data)

In [6]:
train_idx=int(data.shape[0]*0.85)
dev_idx=int(data.shape[0]*0.05)
test_idx=int(data.shape[0]*0.1)

train_data_set=data[0:train_idx,:]
test_data_set=data[train_idx:train_idx+test_idx,:]
dev_data_set=data[train_idx+test_idx:train_idx+test_idx+dev_idx,:]

# Make relu class

In [7]:
class relu():
    def __init__(self):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x <= 0)
    
        out = x.copy()
        out[(x <= 0).any()] = 0
        
        return out
    
    def backward(self,mask, dout):
        mask = (x <= 0)
        dout[mask]= 0
        
        dx = dout
        
        return dx

### MLP

In [8]:
class MLP():
    def __init__(self):
        self.a = 0
        
    def forward(self,W, X, N, b):
        #print('X')
        #print(X)
        
        #print('W')
        #print(W)
        
        #print('next_X')
        #relu
        self.a = np.dot(W,X) + b
        
        return self.a

# Make softmax class

In [9]:
class softmax():    
    def __init__(self):
        self.r = 0
        
    def forward(self,X,h,U,d):
        self.r = np.dot(U, h)+d

        #print('R')
        #print(self.r)

        delta = np.max(self.r)
        
        #print('delta')
        #print(delta)
        self.exp_o = np.exp(self.r-delta)
        self.sum_exp_o = np.sum(self.exp_o)
        
        #print('\nexp_o')
        #print(self.exp_o)
        
        #print('\nsum_exp_o')
        #print(self.sum_exp_o)
        
        #print('\nsoftmax')
        self.softmax = self.exp_o / self.sum_exp_o
        
        return self.softmax

In [10]:
# Make Cross_entropy class

In [11]:
class CE():
    def __init__(self):
        self.result = 0
        
    def forward(self, y, softMAX):
        # cross_entropy + 마이너스 무한대가 생기지 않도록 만듦
        self.result = - np.sum(y * np.log(softMAX))
        print('Loss')
       
        return self.result
        
    def backward(self, y, softMAX):
        batch_size = y.shape[0]
        dJdR = (softMAX-y) / batch_size
        
        #print('dx')
        #print(dJdR)
        
        return dJdR

# 4.4

In [12]:
model = MLP()
ce = CE()
Relu = relu()
SOFTMAX = softmax()

layer=[]
param_w = []
param_b = []
affine = []

grad_w = []
grad_b = []
grad_affine = []


L = int(input('Layer: '))

for i in range(L):
    print(i+1,'th Dimenson', end = ' ') 
    d=int(input(':'))
    
    layer.append(d)
    
length = len(layer)

minibatch_size = int(input('Minibatch size(10~500): '))
epoch_size = int(input('epoch size: '))

X=data[:,0]
number_minibatch= np.int(np.ceil(X.shape[0]/minibatch_size))


def learning(data, idx, minibatch_size, epoch_size):
    for e in range(1,epoch_size+1):
        b = np.random.randint(-4,4,size=1)
        d = np.random.randint(-4,4,size=1)
        if (e==1):
            print('*************',e,'번차 epoch *************')
            data=np.random.permutation(data)
            X_batch= data[:,0]
            y_batch= data[:,1]
            X_batch=np.reshape(X_batch,(idx,1))
            y_batch=np.reshape(y_batch,(idx,1))

            number_minibatch= np.int(np.ceil(X_batch.shape[0] / minibatch_size))

        #    assert X_batch.shape[0] == y_batch.shape[0]
        #    assert X_batch.shape[1] == weights['W'].shape[0]

            for n in range(1, number_minibatch+1):
                X_batch_temp=X_batch[minibatch_size*n-(minibatch_size-1)-1:minibatch_size*n]
                y_temp=y_batch[minibatch_size*n-(minibatch_size-1)-1:minibatch_size*n]

    ########################## MLP ########################## 
                for i in range(length):
                    print('-------------',i+1,'번째 layer-------------')
                    if (i == 0):
                        x = X_batch_temp
                        W = np. random.randint(-4,4,(layer[i],np.array(x).shape[0]))
                        a = model.forward(W, x,layer[i],b)
                        #print(a)

                        x = Relu.forward(a)
                        param_w.append(W)
                        param_b.append(b)
                        affine.append(a)
                    else:
                        W = np. random.randint(-4,4,(layer[i],np.array(x).shape[0]))
                        a = model.forward(W, np.array(x),layer[i],b)
                        Relu.forward(a)
                        #print(a)

                        param_w.append(W)
                        param_b.append(b)
                        affine.append(a)
                        x = Relu.forward(a)

                U = np. random.randint(-4,4,(X.shape[0], x.shape[0]))

                o = SOFTMAX.forward(X,x,U,d) # softmax

                print('J')
                J = ce.forward(y_temp, o) # cross_Entropy
                print(J)

                ############# BACKWARD #############
                dJdR = ce.backward(y_temp,o) #softmax & cross_Entropy loss
                dJdU = np.dot(dJdR,np.transpose(x))
                dJdh = np.dot(np.transpose(U), dJdR)
                dJda = Relu.backward(affine[2],dJdh)

                param_w.reverse()
                affine.reverse()

                affine.append(X)
                for x in range(length):
                    print('------------- from back',x+1,'번째 backpropagation-------------')
                    if (x == 0):
                        dJd_ = dJda
                    else :
                        dJd_ = np.dot(np.transpose(param_w[x-1]),dJd_)
                    dJdw = np.dot(dJd_, np.transpose(affine[x+1]))
                    dJdb = np.dot(dJd_, np.ones_like(model.b))

                    #print('dJdw')
                    #print(dJdw)
                    grad_w.append(dJdw)
                    grad_b.append(dJdb)

                for key in range(length):
                    param_w[key]=param_w[key]- 0.00001 * grad_w[key]
                    param_b[b]=param_b[b]- 0.00001 * grad_w[b]
                U = U - 0.00001 * dJdU
                d = d - 0.00001 * dJdd

        else:
            for n in range(1, number_minibatch+1):
                X_batch_temp=X_batch[minibatch_size*n-(minibatch_size-1)-1:minibatch_size*n]
                y_temp=y_batch[minibatch_size*n-(minibatch_size-1)-1:minibatch_size*n]

    ########################## MLP ########################## 
                for i in range(length):
                    print('-------------',i+1,'번째 layer-------------')
                    if (i == 0):
                        x = X_batch_temp
                        a = model.forward(param_w[n], np.array(x),layer[n],param_b[n])
                        #print(a)

                        x = Relu.forward(a)
                        param_w.append(W)
                        affine.append(a)
                    else:
                        a = model.forward(param_w[n], np.array(x),layer[n])
                        Relu.forward(a)
                        #print(a)

                        param_w.append(W)
                        affine.append(a)
                        x = Relu.forward(a)

                U = np. random.randint(-4,4,(X.shape[0], x.shape[0]))
                o = SOFTMAX.forward(X,x,U,d) # softmax

                print('J')
                J = ce.forward(y_temp, o) # cross_Entropy
                print(J)

                ############# BACKWARD #############
                dJdR = ce.backward(y_temp,o) #softmax & cross_Entropy loss
                dJdU = np.dot(dJdR,np.transpose(x))
                dJdh = np.dot(np.transpose(U), dJdR)
                dJda = Relu.backward(affine[2],dJdh)

                param_w.reverse()
                param_b.reverse()
                affine.reverse()

                affine.append(X)
                for x in range(length):
                    print('------------- from back',x+1,'번째 backpropagation-------------')
                    if (x == 0):
                        dJd_ = dJda
                    else :
                        dJd_ = np.dot(np.transpose(param_w[x-1]),dJd_)
                    dJdw = np.dot(dJd_, np.transpose(affine[x+1]))
                    dJdb = np.dot(dJd_, np.ones_like(param_b[x]))

                    #print('dJdw')
                    #print(dJdw)
                    grad_w.append(dJdw)
                    grad_b.append(dJdb)

                for key in range(length):
                    param_w[key]=param_w[key]- 0.00001 * grad_w[key]
                    param_b[b]=param_b[b]- 0.00001 * grad_w[b]
                U = U - 0.00001 * dJdU
                d = d - 0.00001 * dJdd
train_ = learning(train_data_set,train_idx,minibatch_size, epoch_size)

Layer:  3


1 th Dimenson 

: 3


2 th Dimenson 

: 2


3 th Dimenson 

: 3
Minibatch size(10~500):  100
epoch size:  3


************* 1 번차 epoch *************
------------- 1 번째 layer-------------


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()